# Plot Attrib Perf and historical risk

In [1]:
# import standard libs
from IPython.display import display
#from IPython.core.debugger import set_trace as bp
#import datetime

# import in-house lib
import updPosesHT4 as upd

# import python scientific stack
import pandas as pd
import numpy as np
#import datetime

import matplotlib.pyplot  as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('talk')

# import visual tools
#import bqplot as bq
#import plotly.graph_objs as go
#import plotly
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)

# import util libs
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
%matplotlib inline

Toggle output scrolling of selected cells

In [2]:
%%javascript
require(
    ["notebook/js/outputarea"],
    function (oa) {
    oa.OutputArea.auto_scroll_threshold = -1;
    console.log("Setting auto_scroll_threshold to -1");
    });

<IPython.core.display.Javascript object>

In [12]:
start_date, end_date = pd.datetime(2017,1,1), pd.datetime.today()

In [13]:
print('Loading Data from {:%d,%b %Y} to {:%d,%b %Y}'.format(start_date, end_date))
df = upd.getPosesRpt(start_date, end_date, 1.0).reset_index()
df = df.drop(df[df.PM=='PTF'].index)

Loading Data from 01,Jan 2017 to 16,Jul 2018


In [5]:
def wSelectMultiple(df, fields):
    return {fld: widgets.SelectMultiple(options= ['ALL'] + elts, 
                                        value= ['ALL'],
                                        rows= min(len(elts)+1, 8),
                                        description= fld)
                    for fld, elts in 
                        ((fld, list(df[fld].sort_values().unique())) for fld in fields)}

In [14]:
flds = ['Asset', 'PM', 'PtfId', 'Code']
wSel = wSelectMultiple(df.reset_index(), flds)
wSel['FundReference'] = widgets.ToggleButtons(options= ['DGA', 'ITX3'])
wFields = widgets.ToggleButtons(options=['pl', 'rv', 'dir', 'deltaAUM', 'gammaAUM', 'vegaAUMbp', 'thetaAUMbp', 'bpvEur'],
    description='Fields',
    tooltips=['PL total in bps', 'PL Relative Value in bps of slow', 'PL eq. benchmark in bps'],)

In [15]:
def update_Code(*args):
    #print(args)
    codes = list(df[df['Asset'].isin(list(args[0]["new"]))].Code.sort_values().unique())
    #codes = list(df.query(f'Asset in {list(args[0]["new"])}').Code.unique())
    wSel['Code'].options = ['ALL'] + codes
    wSel['Code'].value = ['ALL']   
wSel['Asset'].observe(update_Code, 'value')

In [16]:
def plot_filtered(field= wFields, **kwargs):
    global select
    select = df
    flds = {fld: values for fld, values in kwargs.items() if not 'ALL' in values}
    #print(flds)
    for fld, values in flds.items():
        if isinstance(values, str):
            values = [values]
        if 'mask' in locals(): 
            mask = mask & df[fld].isin(values)
        else:
            mask = df[fld].isin(values)
    if 'mask' in locals():
        select = df[mask]
        gp = ['PtfDate'] + list(flds.keys())
    else:
        gp = ['PtfDate']
    select = select.groupby(gp)[wFields.value].sum()
    if flds:
        select = select.unstack(0).T
        if wFields.value in ['pl', 'rv', 'dir']:
            select= select.cumsum()
    if select.empty or not select.any().any():
        plt.gca().remove();
    else:
        ax= select.plot(figsize= (16, 9), title= wFields.value)
        ax.yaxis.tick_right()
        ax.hlines(0, *ax.xaxis.get_data_interval(), linewidth = 2.0, color='r');
        
    return kwargs
w = widgets.interactive(plot_filtered, **wSel)

In [17]:
display(VBox([wFields, 
              widgets.Label('Please select filters'),
              #HBox(w.children[:-1]), 
              HBox(list(wSel.values())), 
              w.children[-1]]))